In [ ]:
!pip install -q selenium
!apt install -qq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
urls = {
  "primary": "https://overframe.gg/tier-list/primary-weapons/",
  "secondary": "https://overframe.gg/tier-list/secondary-weapons/",
  "melee": "https://overframe.gg/tier-list/melee-weapons/"
} 

In [4]:
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.get(urls["primary"])
print(wd.title)

Warframe Primary Weapon Tier List 2022 - Best Primary Weapons - Overframe


In [ ]:
from selenium.webdriver.common.by import By

tierContainers = wd.find_elements(by=By.CLASS_NAME, value="TierList_tierContainer__psIt4")

wStorage = []
tStorage = []

for tierContainer in tierContainers:
  currTier = tierContainer.find_element(By.CLASS_NAME, "TierList_tierTitle__PdTC9").text
  weapons = tierContainer.find_elements(By.TAG_NAME, "figcaption")
  for weapon in weapons:
    wStorage.append(weapon.text)
    tStorage.append(currTier)
  
# print(tStorage)
# print(wStorage)

In [14]:
import pandas as pd

d = {
    "weapon": wStorage,
     "tier": tStorage,
}

df = pd.DataFrame(data=d)
df

,weapon,tier
0,KUVA ZARR,S Tier - Prime Time
1,NATARUK,S Tier - Prime Time
2,PROBOSCIS CERNOS,S Tier - Prime Time
3,KUVA BRAMMA,S Tier - Prime Time
4,CEDO,S Tier - Prime Time
...,...,...
153,LATRON,D Tier - Weak Picks
154,BRATON,D Tier - Weak Picks
155,HIND,D Tier - Weak Picks
156,DERA,D Tier - Weak Picks


### SQL Part

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)